In [6]:
import pandas as pd
from datetime import datetime, timedelta, time
import numpy as np
import seaborn as sns
import sys


print("Python Version:", sys.version)
print("Pandas Version:", pd.__version__)
print("Numpy Version:", np.__version__)
print("Seaborn Version:", sns.__version__)

main_path = '/Users/PC/Desktop/Insight/MTASafety'

Python Version: 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]
Pandas Version: 0.24.2
Numpy Version: 1.16.2
Seaborn Version: 0.9.0


In [2]:
# spherical distance
# Taken from here: 
# https://community.esri.com/groups/coordinate-reference-systems/blog/2017/10/05/haversine-formula
def haversine(coord1: object, coord2: object):
    import math

    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    meters = round(meters, 3)
    km = round(km, 3)
    miles = round(meters/1609.34,3)
    
    return miles
    #print(f"Distance: {meters} m")
    #print(f"Distance: {km} km")

In [180]:
Crimes=pd.read_csv(main_path + '/CrimeData/CrimeData2.csv')
Stations=pd.read_csv(main_path + '/RiderData/TurnstileStationsMapping6.csv')

In [100]:
list(zip(Stations['Longitude'],Stations['Latitude']))

[(-73.99107, 40.730053999999996),
 (-74.00019300000001, 40.718803),
 (-73.98384899999999, 40.761728000000005),
 (-73.97499915, 40.68086214),
 (-73.89488591, 40.66471445),
 (-73.90087, 40.884667),
 (-73.95806671, 40.80058156),
 (-73.940859, 40.679919),
 (-73.89878838, 40.74971953),
 (-73.92901818, 40.75196004),
 (-73.9874094, 40.71830606),
 (-73.89165773, 40.67802821),
 (-73.879626, 40.68152),
 (-73.844435, 40.695166),
 (-73.98177094, 40.69064812),
 (-73.82758075, 40.58326844),
 (-73.81365140000001, 40.58809156),
 (-73.89175225, 40.82998745),
 (-73.89661738, 40.82214213),
 (-73.900741, 40.856093),
 (-73.910136, 40.8459),
 (-73.918432, 40.833769000000004),
 (-73.845625, 40.754622),
 (-73.869527, 40.749145),
 (-73.83003000000001, 40.7596),
 (-73.83256899999999, 40.84681),
 (-73.926138, 40.810476),
 (-73.85122199999999, 40.834255),
 (-74.004131, 40.713065),
 (-73.847036, 40.836488),
 (-73.97671329999999, 40.75180743),
 (-73.982076, 40.746081),
 (-73.95107, 40.785672),
 (-73.959874, 40.7736

In [181]:
#Creating Long/Lat Lists
Stations['LONG/LAT']=list(zip(Stations['Longitude'],Stations['Latitude']))
Crimes['LONG/LAT']=list(zip(Crimes['Longitude'],Crimes['Latitude']))

In [182]:
#Calculating the Matrix Size
Crimes2 = Crimes.reset_index()
num_rows = Crimes2.shape[0]

Stations2 = Stations.reset_index()
num_cols = Stations2.shape[0]


In [185]:
# Creating the distance matrix
dist_mat = np.zeros([num_rows, num_cols])
for i in range(0, num_rows):
    for j in range(0, num_cols):
        dist_mat[i,j] = haversine(Crimes2.loc[i, 'LONG/LAT'], Stations2.loc[j, 'LONG/LAT'])

In [187]:
#Distance DataFrame Column/Row Names
ComplaintNum=list(Crimes['CMPLNT_NUM'])
StationName=list(Stations['Station2'])

In [188]:
#Creating the Distance Dataframe
Distance=pd.DataFrame(dist_mat, index=ComplaintNum, columns=StationName)

In [190]:
#Getting the lowest distance Station
CrimesMatch = Distance.idxmin(axis=1)

In [191]:
CrimesMatch.head()

105108340                 1 AV_L
733844903      SHEEPSHEAD BAY_BQ
766704116         NOSTRAND AV_AC
430820533         HOYT-SCHER_ACG
743759837    DELANCEY/ESSEX_FJMZ
dtype: object

In [192]:
#Getting the distance of the lowest distance Station
CrimesDist = Distance.min(axis=1)

In [193]:
CrimesDist.head()

105108340    0.054
733844903    0.059
766704116    0.046
430820533    0.073
743759837    0.045
dtype: float64

In [194]:
Crimes['Nearest Station']=CrimesMatch.values

In [195]:
Crimes['Distance']=CrimesDist.values

In [196]:
Crimes.head()

,CMPLNT_NUM,CMPLNT_FR_DT,DayofWeek,Month,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,...,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,ORIGINALROW,OBSOLETE_ID,LONG/LAT,Nearest Station,Distance
0,105108340,1/1/2017,Sun,January,21:05:00,1/1/2017,21:15:00,13,1/1/2017,578,...,PATROL BORO MAN SOUTH,1 AVENUE,45-64,ASIAN / PACIFIC ISLANDER,F,1:04,27908.0,"(-73.98256326, 40.73137039)",1 AV_L,0.054
1,733844903,1/1/2017,Sun,January,18:25:00,1/1/2017,18:30:00,61,1/1/2017,344,...,PATROL BORO BKLYN SOUTH,SHEEPSHEAD BAY,25-44,WHITE,M,1:05,27909.0,"(-73.95421551, 40.58739066)",SHEEPSHEAD BAY_BQ,0.059
2,766704116,1/1/2017,Sun,January,10:00:00,1/1/2017,10:10:00,79,1/1/2017,109,...,PATROL BORO BKLYN NORTH,NOSTRAND AVENUE,25-44,BLACK,F,1:06,27911.0,"(-73.94955839, 40.68039861)",NOSTRAND AV_AC,0.046
3,430820533,1/1/2017,Sun,January,5:20:00,1/1/2017,5:30:00,84,1/1/2017,361,...,PATROL BORO BKLYN NORTH,HOYT-SCHERMERHORN,18-24,BLACK,F,1:07,27912.0,"(-73.98615855, 40.68902742)",HOYT-SCHER_ACG,0.073
4,743759837,1/1/2017,Sun,January,4:00:00,NaN,NaN,7,1/1/2017,109,...,PATROL BORO MAN SOUTH,DELANCEY STREET,25-44,WHITE HISPANIC,M,1:08,27913.0,"(-73.98820145, 40.71856407)",DELANCEY/ESSEX_FJMZ,0.045


In [198]:
Crimes.to_csv(main_path + '/CrimeData/CrimeData6.csv')

In [199]:
Distance.to_csv(main_path + '/CrimeData/DistanceMatrix2.csv')

In [213]:
#Based on just geolocation how many crimes happen "outside" of a 1/8th of a mile from a subway station
Crimes.query('Distance > .125').shape
#Handcoding the errors

(2059, 42)